# Phase 1: Synthetic Data Generation

In [1]:
!pip install pandas numpy faker


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker()
Faker.seed(42)

# 1. Create Product Dimension
categories = ['Technology', 'Home', 'Food', 'Textiles']
product_data = [] # Cambiado a product_data

for i in range(1, 51):  # 50 unique products
    product_data.append({
        'product_id': i,
        'product_name': f'Product_{i}',
        'category': random.choice(categories), # Corregido spelling de 'category'
        'unit_cost': round(random.uniform(10, 500), 2),
        'selling_price': round(random.uniform(600, 1000), 2)
    })

df_products = pd.DataFrame(product_data)

# 2. Create Sales Facts (Simulating 2 years of data)
sales_data = []
for _ in range(5000):  # 5,000 transactions
    sales_data.append({
        'sale_id': fake.uuid4(),
        'date': fake.date_between(start_date='-2y', end_date='today'),
        'product_id': random.randint(1, 50),
        'quantity': random.randint(1, 10),
        'client_region': fake.country()
    })

df_sales = pd.DataFrame(sales_data)

# Save to CSV for later SQL ingestion
df_products.to_csv('dim_products.csv', index=False)
df_sales.to_csv('fact_sales.csv', index=False)

print("Files generated successfully!")

Files generated successfully!


In [3]:
!pip install sqlalchemy psycopg2


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from sqlalchemy import create_engine

# 1. Set up the connection
# Change 'your_password_here' to the password you use in pgAdmin
user = 'postgres'
password = 'your_password_here'
host = 'localhost'
port = '5432'
db = 'supply_chain_db'

url = f'postgresql://{user}:{password}@{host}:{port}/{db}'
engine = create_engine(url)

# 2. Upload the data to the tables you created
try:
    df_products.to_sql('dim_products', engine, if_exists='append', index=False)
    df_sales.to_sql('fact_sales', engine, if_exists='append', index=False)
    print("✅ Success! Data has been uploaded to PostgreSQL.")
except Exception as e:
    print(f"❌ Error: {e}")

❌ Error: (psycopg2.errors.UniqueViolation) llave duplicada viola restricción de unicidad «dim_products_pkey»
DETAIL:  Ya existe la llave (product_id)=(1).

[SQL: INSERT INTO dim_products (product_id, product_name, category, unit_cost, selling_price) VALUES (%(product_id__0)s, %(product_name__0)s, %(category__0)s, %(unit_cost__0)s, %(selling_price__0)s), (%(product_id__1)s, %(product_name__1)s, %(category__1)s ... 4893 characters truncated ... (product_id__49)s, %(product_name__49)s, %(category__49)s, %(unit_cost__49)s, %(selling_price__49)s)]
[parameters: {'selling_price__0': 933.39, 'product_id__0': 1, 'category__0': 'Food', 'unit_cost__0': 323.35, 'product_name__0': 'Product_1', 'selling_price__1': 648.32, 'product_id__1': 2, 'category__1': 'Home', 'unit_cost__1': 413.86, 'product_name__1': 'Product_2', 'selling_price__2': 839.91, 'product_id__2': 3, 'category__2': 'Home', 'unit_cost__2': 39.84, 'product_name__2': 'Product_3', 'selling_price__3': 663.48, 'product_id__3': 4, 'category